# Load Data for Testing

In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

data_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
            'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/'
            'quickstarts/connected-car-data/connected-car_components.csv')

# Load the car components labeled data
car_components_df = pd.read_csv(data_url)
components = car_components_df["text"].tolist()
labels = car_components_df["label"].tolist()

maxlen = 100                                               
max_words = 10000      

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(components)
sequences = tokenizer.texts_to_sequences(components)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)

indices = np.arange(data.shape[0])                     
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

Using TensorFlow backend.


Found 65 unique tokens.


## Prepare Test Data

In [2]:
test_size = 10

x_test = data[0:test_size]
y_test = labels[0:test_size]

print('Shape of test data tensor:', x_test.shape)
print('Shape of test label tensor:', y_test.shape)

Shape of test data tensor: (10, 100)
Shape of test label tensor: (10,)


# Test Deployment

Provide the **Scoring URI** and **API Key** for the deployed webservice

In [3]:
web_service_url = 'http://13.79.234.6:80/api/v1/service/compliance-classifier-service/score' # provide the scoring URI
api_key = 'LROhFagZQKYDViSDepLPpre44p0wIF0Q' # provide API key

## Make HTTP calls to test the deployed Webservice

In [4]:
import requests
import json

headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}
response = requests.post(web_service_url, json.dumps(x_test.tolist()), headers=headers)
print('Predictions')
print(response.text)
print('Labels')
print(y_test)

Predictions
"[0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0]"
Labels
[0 1 0 1 1 1 0 1 1 1]


In [12]:
import ast
pred = ast.literal_eval(response.text.replace("\"",""))

In [14]:
hit = 0
for x,y in zip(pred, y_test):
    if (int(x)==int(y)):
        hit += 1
print(f"hit ratio: {hit}/{len(pred)} = {hit/len(pred)}")

hit ratio: 10/10 = 1.0
